# TP noté du lundi 08 avril 2019 - Thomas DAILLY

#### Question 1

In [6]:
import pymongo

connexion = pymongo.MongoClient("mongodb://193.51.82.104:2343/")

donnees = connexion.test

#### Question 2

Dans un premier temps, nous regardons la liste des collections présentes puis, dans un second temps, nous appliquons la fonction `count()` à la collection correspondant aux restaurants afin d'en déterminer le nombre exact.

In [8]:
donnees.collection_names()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['restaurants', 'test', 'grades']

In [9]:
# On applique donc "count()" à la collection "restaurants"

donnees.restaurants.count()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  This is separate from the ipykernel package so we can avoid doing imports until


25359

Le nombre exact de restaurants est donc de 25 359.

#### Question 3

On va maintenant chercher à lister les différents quartiers représentés ainsi que les différents types de cuisine. Pour cela, nous utilisons la fonction `distinct()` qui nous permet de lister les valeurs prises par un champ dans une collection :

In [10]:
donnees.restaurants.distinct("borough")

['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Missing']

In [12]:
donnees.restaurants.distinct("cuisine")

['Bakery',
 'Hamburgers',
 'Irish',
 'American',
 'Jewish/Kosher',
 'Delicatessen',
 'Ice Cream, Gelato, Yogurt, Ices',
 'Chinese',
 'Chicken',
 'Turkish',
 'Caribbean',
 'Donuts',
 'Sandwiches/Salads/Mixed Buffet',
 'Bagels/Pretzels',
 'Continental',
 'Pizza',
 'Italian',
 'Steak',
 'Polish',
 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
 'German',
 'French',
 'Pizza/Italian',
 'Mexican',
 'Spanish',
 'Café/Coffee/Tea',
 'Tex-Mex',
 'Pancakes/Waffles',
 'Soul Food',
 'Seafood',
 'Hotdogs',
 'Greek',
 'Not Listed/Not Applicable',
 'African',
 'Japanese',
 'Indian',
 'Armenian',
 'Thai',
 'Chinese/Cuban',
 'Mediterranean',
 'Korean',
 'Bottled beverages, including water, sodas, juices, etc.',
 'Russian',
 'Eastern European',
 'Middle Eastern',
 'Asian',
 'Ethiopian',
 'Vegetarian',
 'Barbecue',
 'Egyptian',
 'English',
 'Other',
 'Sandwiches',
 'Portuguese',
 'Indonesian',
 'Chinese/Japanese',
 'Filipino',
 'Juice, Smoothies, Fruit Salads',
 'Brazilian',
 'Afghan'

#### Question 4

Pour la suite de ce TP, nous allons nous appuyer sur la fonction `affiche()` créée lors de la séance 6 afin d'afficher plus facilement les résultats souhaités.

In [22]:
import pprint

def affiche(res):
    pprint.pprint(list(res))

In [24]:
resultat = donnees.restaurants.find({"cuisine" : "Soups"}, { "name": 1, "borough": 1, "_id": 0})
affiche(resultat)

[{'borough': 'Manhattan', 'name': 'Wichcraft Express'},
 {'borough': 'Staten Island', 'name': 'Original Soupman Of Staten Island'},
 {'borough': 'Manhattan', 'name': 'The Original Soupman'},
 {'borough': 'Manhattan', 'name': 'Peasant Stock'}]


#### Question 5

Commençons tout d'abord par trouver l'ensemble de tous les restaurants ayant eu au moins une fois le grade " Z " et stockons-les tous dans une variable `gradeZ`.

In [40]:
gradeZ = donnees.restaurants.find({"grades.grade": 'Z'})

Dès à présent, nous pouvons appliquer la fonction `count()` à notre variable `gradeZ` ci-dessus afin d'obtenir le nombre total de restaurants ayant eu au moins une fois ce grade " Z ".

In [41]:
gradeZ.count()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


1337

On peut donc en conclure que 1 337 restaurants ont reçu au moins une fois le grade " Z ".

#### Question 6

Pour répondre à cette question, nous allons devoir calculer des agrégats, avec la fonction `aggregate()`.

In [167]:
nb_restaurants = donnees.restaurants.aggregate([
    { "$group": {
        "_id": "$borough",
        "nombre total de restaurants": {"$sum": 1}
    }}
])

affiche(nb_restaurants)

[{'_id': 'Missing', 'nombre total de restaurants': 51},
 {'_id': 'Staten Island', 'nombre total de restaurants': 969},
 {'_id': 'Brooklyn', 'nombre total de restaurants': 6086},
 {'_id': 'Bronx', 'nombre total de restaurants': 2338},
 {'_id': 'Queens', 'nombre total de restaurants': 5656},
 {'_id': 'Manhattan', 'nombre total de restaurants': 10259}]


On obtient alors la liste du nombre de restaurants pour chaque quartier.

#### Question 7

In [166]:
cinq_types = donnees.restaurants.aggregate([
    { "$group": { "_id": "$cuisine", "nb": { "$sum": 1 }} },
    { "$sort": { "nb": -1 }}
])
affiche(cinq_types)

[{'_id': 'American', 'nb': 6183},
 {'_id': 'Chinese', 'nb': 2418},
 {'_id': 'Café/Coffee/Tea', 'nb': 1214},
 {'_id': 'Pizza', 'nb': 1163},
 {'_id': 'Italian', 'nb': 1069},
 {'_id': 'Other', 'nb': 1011},
 {'_id': 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
  'nb': 850},
 {'_id': 'Japanese', 'nb': 760},
 {'_id': 'Mexican', 'nb': 754},
 {'_id': 'Bakery', 'nb': 691},
 {'_id': 'Caribbean', 'nb': 657},
 {'_id': 'Spanish', 'nb': 637},
 {'_id': 'Donuts', 'nb': 479},
 {'_id': 'Pizza/Italian', 'nb': 468},
 {'_id': 'Sandwiches', 'nb': 459},
 {'_id': 'Hamburgers', 'nb': 433},
 {'_id': 'Chicken', 'nb': 410},
 {'_id': 'Ice Cream, Gelato, Yogurt, Ices', 'nb': 348},
 {'_id': 'French', 'nb': 344},
 {'_id': 'Delicatessen', 'nb': 321},
 {'_id': 'Jewish/Kosher', 'nb': 316},
 {'_id': 'Indian', 'nb': 316},
 {'_id': 'Asian', 'nb': 309},
 {'_id': 'Thai', 'nb': 285},
 {'_id': 'Juice, Smoothies, Fruit Salads', 'nb': 273},
 {'_id': 'Korean', 'nb': 262},
 {'_id': 'Sandwiches/Salads/Mixed B

On obtient ainsi la liste des types de cuisine les plus présentes dans New-York dans l'ordre décroissant.

#### Question 8

On s'intéresse à chaque chaque grade en donnant le nombre d'évaluation avec ce grade, ainsi que son score moyen, minimum et maximum.

In [196]:
nos_grades = donnees.restaurants.aggregate([{
    "$group": { "_id": "$grades.grade", "nb": { "$sum": 1 },
    "score_moyen": { "$avg": 1 },
    "score_min": {"$min": 1},
    "score_max": {"$max": 1}}
}])
affiche(nos_grades)

[{'_id': ['Z', 'A', 'P'],
  'nb': 2,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['C', 'P'],
  'nb': 2,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'P'],
  'nb': 7,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'A', 'P'],
  'nb': 4,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['C', 'B', 'P'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'A', 'C', 'P'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'B', 'P', 'C'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'A', 'P', 'B', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'C', 'A'],
  'nb': 3,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'C', 'P'],
  'nb': 4,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['C', 'B', 'A'],
  'n

 {'_id': ['Z', 'B', 'B', 'C', 'C', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'B', 'B', 'B', 'C', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'P', 'B', 'C'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'A', 'C', 'A', 'P', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'B', 'B', 'C', 'B', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'A', 'P', 'A', 'C', 'A', 'A', 'P'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'A', 'B', 'B', 'P', 'B', 'B', 'B'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'P', 'B', 'B', 'A', 'B', 'B'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'B', 'A', 'C', 'B', 'A', 'P'],
  'nb': 1,
  'score_max': 1,
  

  'score_moyen': 1.0},
 {'_id': ['B', 'A', 'A', 'A', 'A', 'P'],
  'nb': 2,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'B', 'A', 'A', 'B', 'A'],
  'nb': 3,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'C', 'A', 'A', 'A', 'A', 'C'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'A', 'C', 'B', 'C'],
  'nb': 4,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'C', 'A', 'A', 'B', 'P', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'P', 'B', 'P'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'A', 'A', 'B', 'C', 'B', 'C'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'B', 'C', 'A', 'C'],
  'nb': 4,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'C', 'C', 'C', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min':

  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'B', 'B', 'A', 'A', 'A', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['C', 'A', 'B', 'A', 'A', 'A'],
  'nb': 2,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'A', 'A', 'P', 'A', 'A'],
  'nb': 2,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'B', 'A', 'A', 'A', 'A'],
  'nb': 16,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['C', 'A', 'A', 'B', 'C', 'B'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'B', 'B', 'A', 'A', 'A', 'B'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'P', 'B', 'C'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'B', 'B', 'B', 'P', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'B', 'A', 'B', 'B', 'P', 'A'],
  'nb': 1,
  'score_max'

  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'P', 'B', 'B', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'C', 'A', 'P'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'B', 'B', 'C', 'A', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'C', 'C', 'P', 'A', 'B', 'C', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'B', 'B', 'A', 'B'],
  'nb': 6,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'B', 'A', 'B', 'A'],
  'nb': 49,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'A', 'B', 'A', 'A', 'C'],
  'nb': 3,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'A', 'A'],
  'nb': 111,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'A', 'C', 'P'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'sc

  'nb': 5,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'A', 'B', 'A', 'A'],
  'nb': 20,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'C', 'P', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'A', 'C', 'B', 'A', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'A', 'C', 'A', 'B'],
  'nb': 14,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'A', 'C', 'A', 'A', 'A'],
  'nb': 3,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['C', 'A', 'A', 'A', 'A'],
  'nb': 5,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'A', 'C', 'A'],
  'nb': 11,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'A', 'B', 'A', 'A', 'A'],
  'nb': 11,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'C', 'B', 'B', 'B'],
  'nb': 2,
  'score_max': 1

 {'_id': ['A', 'A', 'C', 'P', 'B'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'C', 'B', 'A', 'B', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'A', 'B', 'B', 'P'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'A', 'A', 'C'],
  'nb': 4,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'B', 'C', 'B', 'A', 'P', 'C', 'B'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'B', 'B', 'C', 'C', 'P'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'A', 'B', 'B', 'C', 'C'],
  'nb': 2,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'A', 'A', 'B', 'A', 'B'],
  'nb': 3,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'P', 'A', 'P', 'A', 'A', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.

  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'P', 'A', 'C', 'A', 'A'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'B', 'A', 'A', 'C', 'C'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'A', 'B', 'P', 'A'],
  'nb': 5,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'B', 'B', 'A', 'A', 'A', 'A'],
  'nb': 2,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['B', 'A', 'A', 'C', 'A', 'B'],
  'nb': 2,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'B', 'P', 'B', 'A'],
  'nb': 2,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'C', 'B', 'A'],
  'nb': 2,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['A', 'A', 'A', 'P', 'B', 'C', 'C', 'B'],
  'nb': 1,
  'score_max': 1,
  'score_min': 1,
  'score_moyen': 1.0},
 {'_id': ['Z', 'B', 'A', 'A', 'A', 'C'],
  'nb': 4,
  'score_max': 1,
  'score_

Pour bien répondre à cette question, il aurait fallu réussir à extraire chaque grade indépendamment des autres grades qu'un même établissement a pu recevoir.

#### Question 9

In [199]:
import pandas

data = donnees.restaurants.find()
dt = pandas.DataFrame(list(data))

dt

,_id,address,borough,cuisine,french,grades,name,restaurant_id
0,58ac16d1a251358ee4ee87dd,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,NaN,"[{'date': 2014-03-03 00:00:00, 'grade': 'A', '...",Morris Park Bake Shop,30075445
1,58ac16d1a251358ee4ee87de,"{'building': '469', 'coord': [-73.961704, 40.6...",Brooklyn,Hamburgers,NaN,"[{'date': 2014-12-30 00:00:00, 'grade': 'A', '...",Wendy'S,30112340
2,58ac16d1a251358ee4ee87df,"{'building': '351', 'coord': [-73.985135599999...",Manhattan,Irish,NaN,"[{'date': 2014-09-06 00:00:00, 'grade': 'A', '...",Dj Reynolds Pub And Restaurant,30191841
3,58ac16d1a251358ee4ee87e0,"{'building': '2780', 'coord': [-73.98241999999...",Brooklyn,American,NaN,"[{'date': 2014-06-10 00:00:00, 'grade': 'A', '...",Riviera Caterer,40356018
4,58ac16d1a251358ee4ee87e1,"{'building': '97-22', 'coord': [-73.8601152, 4...",Queens,Jewish/Kosher,NaN,"[{'date': 2014-11-24 00:00:00, 'grade': 'Z', '...",Tov Kosher Kitchen,40356068
5,58ac16d1a251358ee4ee87e2,"{'building': '8825', 'coord': [-73.8803827, 40...",Queens,American,NaN,"[{'date': 2014-11-15 00:00:00, 'grade': 'Z', '...",Brunos On The Boulevard,40356151
6,58ac16d1a251358ee4ee87e3,"{'building': '2206', 'coord': [-74.1377286, 40...",Staten Island,Jewish/Kosher,NaN,"[{'date': 2014-10-06 00:00:00, 'grade': 'A', '...",Kosher Island,40356442
7,58ac16d1a251358ee4ee87e4,"{'building': '7114', 'coord': [-73.9068506, 40...",Brooklyn,Delicatessen,NaN,"[{'date': 2014-05-29 00:00:00, 'grade': 'A', '...",Wilken'S Fine Food,40356483
8,58ac16d1a251358ee4ee87e5,"{'building': '6409', 'coord': [-74.00528899999...",Brooklyn,American,NaN,"[{'date': 2014-07-18 00:00:00, 'grade': 'A', '...",Regina Caterers,40356649
9,58ac16d1a251358ee4ee87e6,"{'building': '1839', 'coord': [-73.9482609, 40...",Brooklyn,"Ice Cream, Gelato, Yogurt, Ices",NaN,"[{'date': 2014-07-14 00:00:00, 'grade': 'A', '...",Taste The Tropics Ice Cream,40356731


A ce stade, les restaurants sont déjà en ligne. Il nous faut maintenant mettre en colonne le score moyen obtenu pour chaque grade possible.